# **download and analyse the 100k mutation and seq data**
- the mut file is 1 based (need to use the site-1 to index) 
- the mut file has this werid column before the "ref" col... sometimes "." and sometimes empty, which fuckup up the line.split() indexing. 
    - need to use the file outputted from this notebook 

**get mutation data**

In [1]:
from Bio import SeqIO
import gzip 
import pandas as pd 
from collections import defaultdict
from tqdm import tqdm 

In [1]:
!wget https://static-content.springer.com/esm/art%3A10.1038%2Fnature24018/MediaObjects/41586_2017_BFnature24018_MOESM2_ESM.zip -P data/

--2021-09-01 13:10:10--  https://static-content.springer.com/esm/art%3A10.1038%2Fnature24018/MediaObjects/41586_2017_BFnature24018_MOESM2_ESM.zip
Resolving static-content.springer.com (static-content.springer.com)... 151.101.184.95
Connecting to static-content.springer.com (static-content.springer.com)|151.101.184.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4005319 (3.8M) [application/zip]
Saving to: ‘data/41586_2017_BFnature24018_MOESM2_ESM.zip’

100%[======================================>] 4,005,319   --.-K/s   in 0.09s   

2021-09-01 13:10:11 (41.3 MB/s) - ‘data/41586_2017_BFnature24018_MOESM2_ESM.zip’ saved [4005319/4005319]



In [4]:
!unzip data/41586_2017_BFnature24018_MOESM2_ESM.zip -d data/

Archive:  data/41586_2017_BFnature24018_MOESM2_ESM.zip
  inflating: data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs.tar.gz  


In [7]:
!gunzip  data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs.tar.gz

**get sequence**

In [10]:
%%bash 
#from "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes" --> chr*.fa.gz: compressed FASTA sequence of each chromosome.
mkdir data/hg38_seq/
for i in {1..22}
do
    wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr$i.fa.gz -P data/hg38_seq/
done   

--2021-09-01 13:16:04--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr1.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74778116 (71M) [application/x-gzip]
Saving to: ‘data/hg38_seq/chr1.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  373K 3m15s
    50K .......... .......... .......... .......... ..........  0%  765K 2m25s
   100K .......... .......... .......... .......... ..........  0%  770K 2m8s
   150K .......... .......... .......... .......... ..........  0% 64.9M 96s
   200K .......... .......... .......... .......... ..........  0% 85.8M 77s
   250K .......... .......... .......... .......... ..........  0%  793K 80s
   300K .......... .......... .......... .......... ..........  0% 31.2M 69s
   350K .......... .......... .......... .......... 

**confirming the required offset (need site-1) & creating the wrangled output file**

In [2]:
mut_lines = open("data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs.tar").readlines()


In [3]:
mut_lines = mut_lines[38:] #remove the header in list object for iteration 

In [8]:
!gzip -r data/hg38_seq/

In [32]:
#following this tutorial for opening zipped files with seqio https://stackoverflow.com/questions/42757283/seqio-parse-on-a-fasta-gz
mismatch = []
dup_muts_count=0
prev_mut_sites = []
with open('data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs_wrangled_cutDown.tsv', 'w') as f:
    f.write('CHROM\tPOS\tREF\tALT\n')
    
    for i in tqdm(range(1,23)): 
        cur_chrom = "chr{}".format(i)

        with gzip.open("data/hg38_seq/{}.fa.gz".format(cur_chrom), "rt") as handle:
            for cur_chrom_seqobj in SeqIO.parse(handle,"fasta"): 

                for mut_line in mut_lines: 


                    if mut_line.split("\t")[0] == cur_chrom: #only want the muts correpsonding tot he currently open seqIO obj 
                    
                        site = int(mut_line.split("\t")[1])
                        line_to_write = [cur_chrom, site]

                        if mut_line.split("\t")[2] ==".": #theres this qweird thing where thr col before the "ref" sometimes has "." in it and sometimes doesnt. 
                            bp_ref_fromMutData = mut_line.split("\t")[3].upper()
                            line_to_write.extend(mut_line.split("\t")[3:5])
                        else: 
                            bp_ref_fromMutData = mut_line.split("\t")[2].upper()
                            line_to_write.extend(mut_line.split("\t")[2:4])
                        bp_fromSeq = cur_chrom_seqobj.seq[site-1].upper()

                        if len(bp_ref_fromMutData)==1: #dont want indels 
                            assert(bp_fromSeq == bp_ref_fromMutData) #testing that the hg38 seq = ref position in the df 

                            #writing to file 
                            line_string = str()
                            for i in range(0,len(line_to_write)): 
                                line_string = line_string+str(line_to_write[i])+"\t"
                            line_string = line_string.rstrip("\t")
                            line_string=line_string+"\n"
                            f.write(line_string)
                            

                            

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [01:00<00:00,  2.75s/it]


In [33]:
! sort -k1,1 -k2,2n data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs_wrangled_cutDown.tsv | uniq > data/nature24018-s2/Aging_Oocytes_Supplementary_Table_DNMs_wrangled_unique.tsv